<a href="https://colab.research.google.com/github/ahcamachod/1893-word2vec-interpretacion-del-lenguaje-humano-con-word-embedding/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color=red>**ONE-HOT-ENCODING**</font>

---



En este notebook desarrollaremos un clasificador de noticias para una start-up de noticias llamada **Alura Latam News**.

El modelo Word2Vec utilizado para entrenar nuestro clasificador fue tomado de la siguiente fuente:

Aitor Almeida, Aritz Bilbao Jayo. (2018) "Word2vec models for the Spanish Language." Available from: https://github.com/aitoralmeida/spanish_word2vec, NO funciono!, en lugar utilize **SBW-vectors-300-min5.bin.gz**

###**EXPLORANDO EL DATASET**

In [ ]:
# 1°) Importamos PANDAS, cargamos los archivos de Entrenamiento y Prueba que ya nos dio el profesor del aula(excel)--->

import pandas as pd

entrenamiento = pd.read_csv("/content/noticias_entrenamiento.csv")
test = pd.read_csv("/content/noticias_prueba.csv")

# Vemos las primeras 5 filas de los dataset.
entrenamiento.sample(5)

,fecha,titulo,pais,extracto,resumen,enlace,categoria
58802,2022-04-03 21:26:27,"En una programación atípica, se disputó la ter...",AR,FÚTBOL LOCAL,(Foto: Bernardo Rolón / prensa Cadetes)\n En u...,https://quedigital.com.ar/deportes/en-una-prog...,deportes
31234,2022-03-29 04:06:00,Salud y pensiones serán resueltas por la ciuda...,DO,En República Dominicana no se garantiza el der...,Millizen Uribe En República Dominicana no se g...,https://hoy.com.do/salud-y-pensiones-seran-res...,ciencia y tecnologia
66467,2022-04-02 06:00:00,La sociedad mexicana,MX,El fiscal incómodo. El sainete dramático que p...,El fiscal incómodo. El sainete dramático que p...,https://www.jornada.com.mx/notas/2022/04/02/ec...,economia
61864,2022-03-31 09:16:54,Georgia.- El presidente de Osetia del Sur dice...,ES,El presidente de la región separatista georgia...,Destaca que podría unirse a Osetia del Norte y...,https://www.notimerica.com/politica/noticia-ge...,politica
81080,2022-03-22 00:00:00,El Gobierno acordó retrotraer precios de los a...,AR,La Secretaría de Comercio Interior que encabez...,El Gobierno afirmó que acordó con supermercado...,https://sinmordaza.com/noticia/265531-el-gobie...,economia


In [ ]:
# Vemos las primeras 5 filas de los dataset.
test.sample(5)

# Nos devuelve los dataset con varias filas y columnas.

,fecha,titulo,pais,extracto,resumen,enlace,categoria
21662,2022-04-02 08:00:00,"Subclases de proteínas, claves en patologías l...",ES,"Guadalupe Sabio, del CNIC, estudia la activaci...",Regístrate gratis en Diario Médico. Para segui...,https://www.diariomedico.com/medicina/endocrin...,ciencia y tecnologia
13542,2022-03-18 16:00:00,Alesi Díaz estrena canción de Farruko a ritmo ...,DO,"El exponente de merengue urbano, Alesi Diaz, p...","Entretenimiento viernes, 18 de marzo de 2022 S...",https://listindiario.com/entretenimiento/2022/...,entretenimiento
20832,2022-03-23 06:57:59,Nueva caída en el precio de la luz: estas son ...,ES,"El precio medio será de 212,44 euros el megava...",En el mes de marzo el precio de la luz no ha p...,https://www.larazon.es/economia/20220323/vs4nl...,economia
2739,2022-03-30 00:00:00,"IPC: La inflación sigue desbocada y sube un 9,...",US,Los augurios se han cumplido y la guerra ha pr...,NaN,https://www.youtube.com/watch?v=PpHOsXZAgFQ,economia
16013,2022-03-23 19:15:48,Muere la ex secretaria de Estado estadounidens...,PA,La exsecretaria de Estado estadounidense Madel...,Asistente Financiero\n\nasistentefinanciero.co...,https://www.prensa.com/mundo/muere-la-ex-secre...,mundo


In [ ]:
# 2°) Vemos los tamaños de los dataset --->

entrenamiento.shape

(91844, 7)

In [ ]:
test.shape

# Los 2 dataset nos da un total de 113.000 noticias para entrenar y validar el modelo.

(22961, 7)

###**ONE-HOT-ENCODING**

Vamos a hablar específicamente sobre el vector de palabras. Entonces, hemos hablado de vector de textos, vector de palabras, que es lo que queremos. Entonces la idea es reproducir nuestras palabras utilizando técnicas de incrustación de palabras también conocidas en inglés, como Word Embedding.
Entonces para poder entender en qué consiste la <font color=orage>Vectorización de palabras</font>, vamos a hablar un poco sobre cómo funciona el aprendizaje de idiomas. Digamos cuando tú estás estudiando otra lengua, por ejemplo el inglés, entonces, quieres aprender a decir, a hacer la pregunta, por ejemplo: ¿usted tiene carro? Entonces, uno aprende que la traducción es: Do you have a car?
Entonces tú comienzas a practicar Do you have a car? Do you have a car? Do you have a car? Do you have a car? Hasta que llega un punto que ya de tanto usar y de tanto ver la misma frase como que se te queda grabada y ya automáticamente, ya sabes cómo preguntar: ¿usted tiene un carro? en inglés, Do you have a car?
Ahora, ¿qué sucede después de haber utilizado tanto esa frase? Supongamos que quieres aprender una nueva frase. Entonces, en vez de preguntar, ¿usted tiene carro? Ahora vas a preguntar, será que ¿ella tiene carro? Entonces, la traducción automáticamente en inglés sería: Does she have a car? Entonces Does she have a car? Does she have a car? Does she have a car?

***Y esa repetición entonces ya empieza a mostrar una especie de patrón***, una semejanza, algo que se torna repetitivo que será entonces lo único que cambia es la parte inicial de la frase o algunas palabras de ella, pero el resto se mantiene igual. Para el caso. ¿Usted tiene un carro? Sería: Do you have a car? Have a car permanece intacto, solo la primera parte de la frase es la que cambia.
En el siguiente caso, ¿ella tiene carro? Does she have a car? Entonces noten ustedes que have a car se mantiene igual y lo único que cambiaría serían estas frases o estas palabras iniciales de la pregunta. Pero ahora Does she have a car? Does she have a car? Does she have a car? ¿Y qué sucede si en algún punto, ella tiene bicicleta?

Después de haber practicado tanto, ya como que nuestro cerebro de alguna forma entiende que ya la pregunta no va a ser: Does she have a car? Does she have a car? Sino Does she have a bike? Entonces se mantiene toda esta primera parte de la pregunta, Does she have a? Y lo único que cambia sería este sustantivo en vez de carro sería bicicleta. Bike.
Entonces así es como nosotros, los seres humanos, vamos también aprendiendo los idiomas. Todo con la repetición y con el uso y con la práctica. ¿Entonces, cómo funciona la estructura de un vector de palabras, digamos?
Nosotros al entender, al lograr comprender la estructura de una frase, lo único que debemos hacer es básicamente intercambiar las palabras porque ya la estructura ya quedó como que de una cierta forma almacenada pues en nuestra memoria, en nuestro cerebro, y por eso podemos elaborar nuevas frases.
Entonces partimos de una frase y a partir de esta frase original podemos crear nuevas frases, varias frases, intercambiando las palabras. Entonces, de esa manera nosotros volvemos nuestro conocimiento más general y al combinar palabras, entonces se obtienen nuevas frases.

Y ahí es donde entramos entonces en el área del <font color=orage>Word Embedding</font>. ***Entonces vectorizar palabras no es otra cosa más que una representación numérica de la palabra. Entonces es simplemente yo tomar una palabra y representarla a través de números en un vector que sería un array de una dimensión y puede tener diversas posiciones.***

Puedo decir que yo represento esa palabra con, digamos, dos números. 3 50, 100, 200, 300, 500, 1000, los números que yo quiera, siempre y cuando los mantenga en una dimensión, entonces por eso se llama vector. Es un vector, la palabra representada, digamos con un número que está dividido o varios números que se encuentran en un vector.
Entonces, retomando aquí tenemos la visualización de las palabras, representación numérica de palabras y para ello vamos a emplear el siguiente **ejemplo**:

Tenga un buen día.

Tenga un excelente día.

Tenga un pésimo día.

Tenemos estas tres frases, estas tres frases van a ser como nuestra base de datos, nuestra base de datos que contiene muchísimas frases, muchísimas palabras, billones de expresiones.
Digamos, imagínate, que tomamos toda Wikipedia en español, tomamos todos los libros que se encuentran en español, tomamos noticias, tomamos reportajes, artículos, libros. Bueno, de todo el contenido que pueda ser posible para poder entonces tener muchísimo, digamos muchísimo, muchísimo contenido.

Entonces esto es lo que se conoce como el ***Corpus***. Por eso es que entrenar un modelo para generar vectores de palabras requiere de muchísimos, muchísimos datos, billones de palabras. Entonces ese es nuestro corpus. Ahora bien, si yo quiero vectorizar estas palabras, entonces lo primero que debo hacer es analizar ese corpus y encontrar las palabras de forma individual, que no se repita.

En estas tres frases tengo tres estructuras, pero hay algo que me las diferencia, que sería lo siguiente:

Tenga un buen día, tengo esa frase y tengo cuatro palabras distintas en la siguiente frase. Solo hay una palabra nueva que yo estoy añadiendo que sería excelente y la siguiente frase sería pésimo.
Entonces estas palabras que no se repiten, se constituyen como mi vocabulario y el <font color=orage>vocabulario entonces va a ser: “tenga un buen, excelente, pésimo día” son las palabras individuales y son estas palabras a las cuales yo debo asignar un valor numérico a cada una de ellas. ¿Entonces de qué forma?

Simplemente le colocó un número, entonces tenga va a ser la número 1, un va a ser la número 2, buen va a ser la número 3, excelente va a ser la número 4, pésimo va a ser la número 5 y día será la número 6, entonces ya tenemos nuestras palabras y cada una tiene un número asociado a ellas.

***¿Qué mecanismo podríamos usar o de qué manera yo puedo entonces emplear esta información disponible para poder asignar o dar una representación numérica a mis palabras?” Entonces, ahí es cuando llega una alternativa, que sería One-Hot-Encoding.***
Simplemente consiste en tomar yo cada una de estas palabras y asignarles un valor a cada una de ellas. Y, por ejemplo, supongamos aquí el 1 es tenga, el 2 es un, el 3 es buen, el 4 es excelente, el 5 es pésimo y el 6 es día. ¿De qué forma yo puedo entonces asignar valores aquí sin que se vayan a repetir?

La forma más sencilla es yo poner cualquier valor aquí dentro, pero que no se me repita dentro de esta tabla. ¿Qué puedo hacer yo? Entonces pensemos un instante. One-Hot-Encoding. Uno solo, una una codificación, digamos, con un caliente, sí. Entonces, ¿qué podríamos hacer? Simplemente asignarle 1.

Va a ser 1 0 0 0 0 0, ese va a ser el vector que representa tenga. Es un vector de 1 x 6, 6 espacios. Un. Entonces, la representación de un va a ser 0 1 0 0 0 0. Buen va a ser 0 0 1 0 0 0. Excelente va a ser 0 0 0 1 0 0. Pésimo va a ser 0 0 0 0 1 0. Y día va a ser 0 0 0 0 0 1. De esa manera, yo estoy garantizando que cada una de mis palabras va a tener un vector diferente.




**Definicion:**

***Analizando One-Hot-Encoding***

Vamos a estudiar algunas formas de representar una palabra como un vector. Una de las representaciones estudiadas se conoce como One-Hot Encoding, que transforma palabras en un vector binario.

***Considerando las frases anteriores, ¿Cuántas dimensiones tendrán los vectores de cada palabra?***

* 6 dimensiones.Tenemos seis palabras diferentes en nuestro “corpus”; de este
  modo, cada palabra se convierte en una dimensión del vector.

###**UTILIZANDO COUNT VECTORIZER**

Vamos a utilizar ONE-HOT-ENCODING utilizando Python.

In [ ]:
# 1°) Importamos el paquete CountVectorizer de la biblioteca Sklearn --->

from sklearn.feature_extraction.text import CountVectorizer

# Instanciamos:
vector = CountVectorizer()

In [ ]:
# 2°) Creamos Variable TEXTO y vamos a colocar una Lista con las frases que vimos en nuestro ejemplo --->

# Corpus:
texto =[
    "tenga un buen dia",
    "tenga un excelente dia",
    "tenga un pesimo dia"
]

In [ ]:
# 3°) Con el TEXTO vamos a hacerle un FIT a nuestro Vector --->

vector.fit(texto)

CountVectorizer()

In [ ]:
# 4°) Si tomamos el Vector ya podemos ver el Vocabulario --->

vector.vocabulary_

# Nos devuelve un Diccionario con valores asignados a cada palabra.

{'tenga': 4, 'un': 5, 'buen': 0, 'dia': 1, 'excelente': 2, 'pesimo': 3}

In [ ]:
# 5°) Si queremos saber como quedo el Vector para la palabra BUEN por ejemplo --->

vector_buen = vector.transform(["buen"])
vector_buen
# Vemos que el VECTOR_BUEN nos dice que es una Matriz Sparse

<1x6 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [ ]:
print(vector_buen)

  (0, 0)	1


In [ ]:
# 6°) Hacemos un print de vector_buen y lo convertimos en un ARRAY --->

print(vector_buen.toarray())

# Nos devuelve la cantidad de palabras del Corpus que NO se repiten con sus posiciones, las 6 posiciones de cada palabra.

[[1 0 0 0 0 0]]


Si agregamos mas palabras al CORPUS el Vocabulario va a aumentar y si seleccionamos otra palabra agregada como Vector en el ARRAY nos va a mostrar la posicion.

Cuando tenemos un CORPUS de 50.000 palabras por ejemplo, nos devolveria 49.000 ceros, y usar one-hot-encoding NO seria lo adecuado usar cuando son muchas palabras, para esto se utilizan otras alternativas para lograr un mejor uso de nuestros Vectores.

#<font color=red>**WORD2VEC PRIMER CONTACTO**</font>

---

###**CONOCIENDO WORD2VEC**

***¿Será que existe alguna técnica que nos permita representar las palabras con un vector de un tamaño fijo que no cambie y aun así, pues tener la posibilidad de evitar que por causa del tamaño de nuestro vocabulario la única alternativa sea One-Hot-Encoding?***

Recordemos que si tengo 50.000 palabras, entonces voy a tener 49999 ceros y solamente un 1. ¿Entonces, cuál será la posibilidad que tenemos? Encontramos entonces, dentro de las muchas técnicas una que se llama <font color=orage>Word2Vec</font>. Entonces vamos representando palabras de acuerdo con el contexto de estas palabras.

Entonces veamos detenidamente en qué consiste Word2Vec. Supongamos que yo tengo el siguiente grupo de palabras: tengo ***Francia, Rusia, Italia, España, Peugeot, Ferrari***. Tengo esas palabras. Entonces al yo tener esas palabras, y tratar de pensar en una representación podríamos tal vez llegar a la conclusión que una alternativa que tenemos sería la de utilizar un ***plano cartesiano.***

Recordemos que el ***plano cartesiano*** es una representación en dos dimensiones de una posición, una determinada posición. Entonces tenemos el eje de las X y el eje de las Y. Entonces, pensemos nuevamente en nuestro vocabulario o en las palabras que tenemos Francia, Rusia, Italia, Ferrari, España, Peugeot. ¿De qué manera yo podría colocar estas palabras dentro de mi plano cartesiano?

Entonces tenemos nuestras palabras nuevamente Francia, Peugeot, España, Rusia, Ferrari, Italia. ¿Cómo yo ubicaría estas palabras en el plano cartesiano? Entonces yo tomaría mi plano cartesiano, el eje de las X y de las Y y colocaría, por ejemplo, Francia en esa posición donde ustedes la ven, Italia, España y un poquito más para la parte superior, Rusia.
Y después, más como para el lado derecho, Peugeot y Ferrari. ***¿Por qué hice eso?*** Nosotros o en mi caso, de tanto escuchar noticias, de haber ido al colegio, de haber ido a la Universidad, de estar continuamente digamos actualizándome con los acontecimientos del mundo, pues sé que por ejemplo, España, Italia y Francia se encuentran en el continente europeo.

Entonces yo hago como esa analogía en mi cabeza, y trato de colocar estas estos nombres de países, sé que son nombres de países como en un lugar. Y como Rusia sé que pertenece a otro continente, lo alejo un poquito, digamos lo primero que ello observé al ver ese vocabulario fue que tenía cuatro países y tenía dos marcas de carros.
Entonces por un lado, pues separé mis países y los coloqué en un lugar. Y por otro lado, también coloque las marcas de carro Ferrari y Peugeot en otro lugar. Pero también noten ustedes, yo sé que Peugeot, también de tanto, pues a mí me gustan los carros y de conocer de carros yo sé que Peugeot es francés y sé que la marca Ferrari es italiana.

Entonces de cierta forma yo también los coloqué en una posición que para mí, que a mi parecer tiene sentido. Entonces esto es lo que me permitió a mí decir, mira, esta es la mejor forma para mí, a mi concepto, de acuerdo con mi forma de ver, cómo quedarían localizadas estas palabras en un plano cartesiano.

<font color=orage>Cuando yo hago esto, entonces estoy utilizando los principios de Word2Vec</font>. Y cuando yo digamos, en vez de utilizar el plano cartesiano tengo las palabras de forma aleatoria en un espacio cualquiera, en un espacio multidimensional, no pensemos ya más en dos dimensiones, sino pensemos, por ejemplo, aquí en este caso en 6 dimensiones, entonces estaríamos hablando de One-Hot-Encoding.

Contemos bien, 1, 2, 3, 4, 5, 6. ¿Por qué dije 6 dimensiones? Porque tengo 6 palabras y recordemos que para poder representar cada palabra, entonces yo utilizaría un 1, y el resto ceros. Entonces esa sería una forma de yo hablar también de One-Hot-Encoding.

Entonces básicamente, así es como funciona Word2Vec.

###**APRENDIZAJE HUMANO**

Pudimos generar una representación vectorial, con una dimensión de 1 por 2, con dos espacios para localizar las palabras de nuestro vocabulario, los países y las marcas de carros.

***¿Entonces de qué forma es que Word2Vec hace estos procesos? Vamos a ver con otro ejemplo.***

Entonces, supongamos que yo tengo la siguiente frase: **Vivo en_________, y al final tengo un país lindo**. Entonces al yo ver esta frase en mi cabeza, asocio que esa palabra que va en blanco, en este espacio en blanco, debe hacer referencia a algún país, entonces yo podría decir, vivo en Rusia, un país lindo, vivo en Francia, un país lindo, vivo en Italia, un país lindo, vivo en España, un país lindo.

O también, por ejemplo, otra frase: conduzco un, tengo un espacio en blanco y rojo, entonces yo ya sé el verbo conducir. Entonces, ¿qué puedo conducir? Puedo conducir un vehículo. Eso es lo primero que viene a mi cabeza entonces podría decir que es un Ferrari, que es un Peugeot, que es un Ford o cualquier marca de carro que ustedes se les pase por su cabeza en ese momento.
Entonces, ¿nosotros cómo llegamos a estas conclusiones? Vuelvo a repetir, como ya tenemos un recorrido por la vida, viendo tanto contenido, viendo televisión, leyendo, estudiando geografía, viendo documentales, películas, y conversando con la gente, entonces, todo este contenido va permitiendo que nuestro cerebro realice estas asociaciones y podamos entonces, a través de un contexto de palabras, poder definir cuál es la palabra que mejor se encaja en una frase.
Entonces, aprendemos nosotros, los seres humanos aprendemos consumiendo contenido, mucho contenido, esa es la forma de aprendernos, así nos culturizamos.

***Entonces, ¿cómo enseñarle el contexto a la máquina? ¿Será que así como nosotros, los seres humanos aprendemos, será que los computadores también pueden aprender?***

Entonces ahí entramos en el área de **NLP o Natural Language Processing o procesamiento del lenguaje humano**, y aquí es donde ya se empiezan a desarrollar todas estas técnicas que lo que hacen es entrenar nuestros Word Embeds, que serían como estos vectores densos para que el computador pueda entonces contextualizar las palabras.






###**ARQUITECTURAS WORD2VEC**
Vamos a hablar un poco sobre la arquitectura de Word2Vec. Entonces, si nosotros queremos que nuestra máquina aprenda digamos de la forma como tal vez nosotros los seres humanos aprendemos, lo primero que vamos a necesitar son datos, muchos y muchos datos, esto vendría siendo el corpus de nuestro Word Embedding.
Entonces necesitamos billones de palabras, muchas, muchas palabras tomadas de diversas fuentes, y mientras más grande el volumen de nuestros datos, también nuestros vectores van a poder generalizar mejor. Vamos a tener una representación vectorial de las técnicas de Word Embedding más precisa.
Vamos a tener digamos un procesamiento de lenguaje natural mucho más cercano a la realidad. Es como nosotros. Mientras más contenido consumimos, mientras más leemos, mientras más nos culturizamos, pues más fácil vamos a poder hablar de diversos temas.
Entonces lo mismo lo hace el computador, entonces lo primero, muchos datos, billones y billones de datos, de palabras, perdón, billones y billones de palabras, mucho contenido. Y todas esas palabras las vamos a alimentar a los algoritmos de Word2Vec.

El algoritmo de Word2Vec es el que se encarga de generar estos vectores. Estos vectores pueden ser de diversos tamaños, como les dije, puede ser de 10, 50, 100, 300, 500, 1000 posiciones, es algo que nosotros mismos somos los que vamos a definirlo, para finalmente obtener entonces nuestro vector de eso que es representado por números, digamos tenemos nuestro vector.

Un array de una dimensión por diversas posiciones, entonces, por ejemplo, tenemos aquí la palabra Ferrari, y tenemos esta representación vectorial de ella y aquí estos tres puntitos nos indica que puede tener el tamaño que nosotros vayamos a definir. Observar esa estructura una entrada digamos, algo que acontece en la mitad y después una salida, ¿qué nos recuerda?

Bueno, para los que ya hicieron el curso de redes neuronales con NumPy, que también está aquí en nuestra plataforma de Alura Latam, entonces vemos aquí que se trata precisamente de una red neuronal. ¿Y cuál es la estructura de la red neuronal? ¿Cómo funciona la red neuronal?

Vamos a abordarlo de una forma un poco general, para que más o menos entendamos cómo es que Word Embedding y, especialmente la técnica de Word2Vec, vamos a hablar de Word2Vec aquí específicamente nuestro curso funciona. Entonces en una red neuronal, nosotros tenemos una capa de entrada, una capa o capas escondidas ocultas y también tenemos una capa de salida.

¿Cómo funciona la red neuronal? Tenemos nuestras capas. Entonces aquí yo tengo mis entradas, que vendría siendo muchos, muchos datos. Sería mi corpus, billones de palabras. Y cada una de estas palabras tiene un peso y un bies asociado a ellas. ¿Cuál peso y cuál bies?
Pues cuando el algoritmo recibe estas palabras por la primera vez, los va a asignar de forma aleatoria, porque así es como funciona la red neuronal, primero asigna pesos y bieses de forma aleatoria, y entonces va recorriendo toda la red, hasta llegar a la salida que vendría siendo el vector denso que contiene la representación numérica de cada una de esas palabras. ¿Pero qué sucede entonces?

Yo tengo una suma de estos pesos y bieses y una función de activación que me permite avanzar por cada capa de mi red neuronal. Y al llegar a la salida entonces lógicamente si yo asigné pesos y bieses de forma aleatoria, lo más seguro es que voy a tener un error muy grande.

Y este error es muy importante porque con este error yo voy a poder generar una función de costo que me va a permitir, digamos, devolverme por mi red y corregir estos pesos y bieses y hacer una serie de iteraciones aquí para de esa manera, entonces, acercarme mejor, tener una mejor aproximación a mi salida.
Entonces, más o menos de forma general, así es como funciona una red neuronal. Entonces, mientras más palabras, mientras más iteraciones, mientras más procesamiento se realice, entonces, mejor va a ser nuestro resultado, vamos a reducir como que toda esa entropía que es ese estado de desorden, de las palabras en general.

Por eso para poder entrenar un modelo de Word Embedding se necesitan de billones de palabras y de mucho procesamiento. Entonces, de esta manera, nosotros ya vamos a comprender mejor cómo es que funciona Word2Vec. Veamos en principio cómo funcionaría CBOW, que sería Continuos Bag of Words, bolsa de palabras continuas sería una traducción literal, bag of words, continuous bag of words. Bueno.

Entonces, supongamos: Vivo en un país lindo. ¿Será que para yo poder definir qué palabra va aquí, necesito todas estas palabras que concluyen, que me llevan a esta frase o será que si yo elimino algunas de esas palabras, aún podría obtener una respuesta aquí?
Entonces, veamos. Si yo quito por ejemplo, una preposición, un artículo indefinido y un adjetivo, que sería en un lindo, si yo quito eso, será que yo aún podría decir qué tipo de palabra llevo aquí. Yo creo que sí, humanamente yo sé que si yo tengo vivo y país lo más seguro es que ahí tenga un país, en ese espacio en blanco, lo que va es un país, porque vivir y país automáticamente me llevan a mí a pensar en un país.

Retomando nuestra analogía de la red neuronal, entonces yo tengo capa de entrada, capa o capas ocultas y capa de salida. Entonces como capa de entrada yo voy a tener mi contexto que se va a hacer la palabra vivo y va a ser la palabra país, y la voy a llevar a mi capa oculta que sería dónde está mi Word2Vec, que aquí es donde aplico las funciones.

Entonces digamos que va a ser Word2Vec, él me va a dar un diversas salidas, me va a decir que es Francia, que es Italia, que es España, en fin, él me va a dar varias salidas, entonces, cuando yo tomo el contexto para determinar la palabra como para establecer el vector denso que representa la palabra que yo necesito colocar en ese espacio en blanco, entonces estamos hablando de la técnica o del ***modelo CBOW, continuous bag of words.***

Ahora bien, si yo no tomo el contexto, sino que tomo la palabra y quiero asumir el contexto, entonces por ejemplo yo tengo a los lados dos espacios en blanco y tengo en la mitad un país, lo más seguro es que yo voy a tratar de asociar otras palabras, trabajo en Francia, que queda en Europa. Vivo en Francia, que es un país bonito. Estudio en Francia.
Bueno, en fin, aquí pueden haber otras palabras que van a contextualizar ese término específico que yo tengo. Entonces, en este caso ya cambia el orden de mi red, ya ahora mi entrada va a ser una única palabra. Entonces voy a entrar a mi capa oculta, que es donde se va a encontrar mi Word2Vec y voy a tratar de obtener la salida que va a ser el contexto en el cual va esta palabra, que eso es lo que yo desconozco.

Entonces, después de hacer todo este entrenamiento, yo al colocar mi entrada, llevarla a mi capa oculta y al obtener mi salida, yo después de varias iteraciones y de bastante aprendizaje, voy a determinar que el mejor contexto para Francia es vivo y país. Y esta otra técnica de Word2Vec es la que se conoce como ***Skip Gram.***

Entonces no tomamos el contexto para asumir la palabra, sino que tomamos la palabra para asumir el contexto y en CBOW tomamos el contexto para descubrir la palabra. Esa es como la diferencia conceptual entre Skip Gram y Word2Vec. ¿Entonces, también en términos técnicos, que se podría decir?

Que Skip Gram generaliza el mejor cuando, por ejemplo, tiene un corpus un poco más reducido. Entonces es más fácil en un algoritmo que se comporta mejor, pero CBOW por el contrario, es más rápido, es más ágil. Ahora bien, TensorFlow dice que no, que CBOW generaliza mejor.

Entonces ya son diversos conceptos, pero la recomendación a nivel general es que se utilicen ambos acercamientos, tanto CBOW como Skip Gram, o sea se utilice tanto el contexto para descubrir la palabra como la palabra para descubrir el contexto, para de esa forma ver cuál modelo trabaja mejor.

###**BUSCANDO MODELOS WORD2VEC**

Debemos cargar un modelo Word2Vec pre entrenado, que haya sido previamente entrenado.
Para poder entrenar un modelo de Word Embedding necesitamos un corpus con millones o con billones de palabras. Y mientras más palabras tengamos, entonces nuestros vectores van a tener una mayor precisión y eso es lo que queremos porque queremos que se acerque mucho a la realidad.

Entonces, para poder hacer ese trabajo, bien sea a través de Deep Learning, bien sea a través de diversas técnicas de Word Embedding, lo importante es que el algoritmo realizado o la forma de realizar todos estos modelos sea fidedigna, sea buena, y para ello necesitamos un recurso confiable.

Hay que buscar en Internet, digamos, hay que ver, por ejemplo Word Embedding en español. Y ustedes digitan, por ejemplo, en Google modelos de Word Embedding en español. Entonces hay diversos modelos, entonces ustedes pueden empezar a buscar y yo al realizar este curso estuve viendo varios modelos, pero realmente solo uno de ellos que encontré fue el que más me llamó la atención. Vamos a ver, aquí en Spanish a ver si lo encuentro.
Uno de ellos, y es este de aquí, de hecho, ya está subrayado. Vamos a darle clic. Entonces, es de estas personas de Aitor Almeida y de Aritz Bilbao. Aquí está la Lafuente. Ellos son Doctores en el tema de procesamiento del lenguaje natural, entonces es un modelo confiable, ya lo probé.

Ustedes pueden inclusive buscar otros modelos, lo importante es que sean modelos de Word2Vec, de embedding, porque estamos en el curso de Word2Vec. Hay otras técnicas como fast text y otras.
Entonces aquí nos da una especificación: modelo son creados usando una ventana de más o menos 5 palabras, descartando aquellas con menos de 5 instancias y creando un vector de 400 dimensiones. Recordemos que nosotros habíamos hablado de que el tamaño del vector lo definíamos nosotros, 50, 100, 200, 300, bueno.

El modelo pre entrenado que vamos a utilizar es de 400 dimensiones. Miren ustedes, el texto utilizado para crear los embeddings ha sido recuperado de noticias Wikipedia, de la BOE española, web scraping y recursos literarios abiertos o sea, muchos datos. El texto usado tiene un total de 3.257.000.000 palabras y 18.852.481.207 caracteres.

3.200.000.000 de palabras y 18.000.000.000 de caracteres. Entonces es algo masivo, es algo enorme. Y bueno, pide que utilicemos la que citemos, pues si vamos a llegar a utilizar estos modelos, a Aitor Almeida, y Aritz Bilbao. Entonces vamos a ir al link externo que ellos nos están dando y nos lleva a una cuenta de GitHub.

Nos dice que los modelos fueron compartidos en este link, de Zenodo, ahorita lo vamos a acceder para descargarlos y aquí también nos tiene una pequeña documentación de cómo cargarlo. Si queremos cargar el modelo completo, entonces vamos a utilizar Word2Vec.load pero también podemos utilizar KeyedVectors, que son vectores con una llave.

Entonces pensemos en un diccionario, tenemos nuestras keys, que serían las palabras, y el vector que sería nuestro valor. Entonces, KeyedVectors para manejar ese concepto. Y vamos específicamente ya a trabajar con KeyedVectors y nuevamente hace la recomendación.

Si usamos nuestros modelos, si usamos los modelos de ellos en nuestros programas, utilizamos la citación de ellos o la cita bibliográfica a la fuente de ese recurso. Entonces vamos a ir a Zenodo.org y aquí nos aparecen Spanish 3B words Word2Vec Embeddings. Entonces, aquí está el modelo completo, el vector completo, bueno, noten ustedes 7.4 gigas, gigante.

Pero vamos a utilizar mejor, digamos el KeyedVectors que tiene 2.9 gigas. Entonces le vamos a dar clic a donde dice download al Keyed_vectors.zip, le dan download y él va a empezar a descargar. Yo, como ya lo descargué, vean, yo no voy a hacer esto porque ya lo descargué. Se va a demorar mucho. Entonces le voy a dar cancelar. Pero ustedes cargan el que dice Keyed_vectors.zip.

Y al tener este Keyed_vectors.zip, entonces lo vamos a subir a nuestro Google Drive. Yo voy a venir a mis downloads, que yo ya lo tengo aquí y lo voy a subir a mi carpeta de Word2Vec. Entonces dice que se demora un poco de minutos, 20 minutos subiendo. Mientras él carga, vamos a estar aquí nuevamente viendo lo que dice estos modelos.

###**CARGANDO EL MODELO**

Acabamos aquí entonces de cargar nuestros Keyed_vectors.zip a drive, pero si queremos conocer un poco mejor cómo es que lo vamos a implementar en Python, volvamos al GitHub de donde encontramos el link de descarga. Ellos trabajan con dos tipos de modelos, gensim full models que sería complete_model.zip, ese no fue el que descargamos y KeyedVectors.
Entonces ese es el archivo que acabamos de descargar, keyed_vectors.zip. Podemos ver las diferencias entre ellos en la siguiente URL, vamos a hacer clic en esta URL, vamos a abrirlo en una nueva guía. Y aquí dice almacenar y consultar word vectors.

Dice aquí: este módulo implementa word vectors, vectores de palabras, y generalmente sets o conjuntos de vectores, keyed by lookup tokens, vectores que se encuentran con una llave a través de una token que vendría siendo el vector, vendría siendo el array de una dimensión con varias posiciones, direccionado hacia una key, que sería la palabra.

Aquí dice que los trained word vectors, que los vectores de palabras entrenados son independientes de la forma como ellos fueron entrenados, Word2Vec, FastText, VarEmbed, diferentes, y ellos pueden ser representados por estructuras solas, como se implementa en este módulo.
Entonces, aquí hacen lo que explica es que la estructura llamada KeyedVectors, es esencialmente un mapeo entre llaves y vectores. Entonces cada vector es identificado por su llave de búsqueda y muy seguidamente, es una token que está en formato de string y esto usualmente se mapea entre sí.
Entonces tenemos la string y dice que nos lleva a un numpy array de una dimensión, o sea al vector. Y queda así como como una especie de diccionario. Entonces, aquí dice: ¿por qué utilizar KeyedVectors en vez de un modelo completo?

Entonces dice que podemos continuar entrenando los vectores en KeyedVectors, dice que no tiene esta posibilidad, no podemos continuar entrenando los vectores. Para poder entrenar o actualizar vectores necesitamos el modelo completo pero todas las demás funcionalidades sí las poseen los KeyedVectors.
Entonces objetos más pequeños entonces dice que el KeyedVector sí lo soporta y el modelo completo no. Dice que los KeyedVectores son más pequeños y necesitan menos memoria RAM porque no necesitan almacenar el estado del modelo que habilita su entrenamiento. Almacenar y cargar del formato nativo de FastText y Word2Vec dice que KeyedVectors también lo hace.

Entonces, los vectores que han sido exportados por Facebook y Google por las herramientas de Facebook y Google, no soportan más entrenamientos, pero los puede cargar a través de KeyedVectors. Entonces vemos que el KeyedVector es muy bueno. Podemos también hacer un append o añadir nuevos vectores.

El KeyedVector lo permite, el modelo completo también. Concurrencia, entonces dice aquí que permite que las consultas concurrentes a vectores se realice, comparten RAM los dos y la forma de cargar dice que utiliza mmap para cargar los datos del disco de forma instantánea. Entonces por eso los KeyedVectors son muy interesantes.

Aquí fue como un resumen. La principal diferencia es que los KeyedVectors no toleran o no soportan un entrenamiento más profundo y aquí, por ejemplo, dice cómo obtener los vectores. Entonces para poder entrenar un modelo completo podemos acceder a la propiedad modelo .wv que es el que mantiene el standalone keyed vectors. ¿Entonces, cómo se haría esto? Entonces se haría importando estos dos módulos e instanciándolos de esta forma.

Ahora bien, si nosotros tenemos ya el keyed vector que se trata del archivo .kv que acabamos nosotros de descargar, entonces la forma de trabajarlo sería así. KeyedVector.load(‘vectors.kv’). Entonces habría que hacer esta importación y ejecutar, instanciar nuestro vector utilizando el comando .load del módulo KeyedVectors, de gensim models.

Si venimos a nuestro notebook, entonces vamos a hacer lo siguiente: from gensim.models import KeyedVectors. Y ahora sí vamos a colocar aquí en nuestro modelo, entonces vamos a llamarlo simplemente modelo y ahí lo que vamos a hacer es KeyedVectors.load y vamos a cargar nuestro archivo kv, ¿pero qué sucede?

Si venimos a nuestro Drive, vean ustedes, yo tengo en mi carpeta MyDrive, yo no tengo el archivo kv directamente sino yo tengo el archivo keyed_vectors.zip. Entonces, ¿qué voy a hacer? Voy a copiar el camino de este archivo y en una celda antes lo que voy a hacer es hacerle un unzip.

Entonces a decir !unzip. Al yo colocar este signo de exclamación, lo que estoy haciendo es usar de una cierta forma la línea de comando de Linux, que funciona atrás de este Colab. Entonces le hacemos unzip. Y él se va a demorar algunos instantes, digamos descomprimiendo el archivo, el complete.kv y el complete.kv.vectors que sería un NumPy array, entonces él los mapea, mapea esos dos archivos.

En el complete.kv se encuentran las keys y en el complete.kv.vectors.npy se encuentran sus respectivos tokens. Entonces, pues él se va a demorar algunos instantes y en cuanto pues se realiza este proceso, entonces nosotros aquí confirmamos que nos haya quedado bien escrito el código que es vector.log y por KeyedVectors. Básicamente esto. Entonces hay que dejar que él se ejecute.

Después de que nuestro modelo esté ejecutado, entonces podemos tratar de ver qué operaciones o qué métodos le podemos aplicar para conocerlo un poco mejor.

#<font color=red>**EXPLORANDO WORD2VEC**</font>

---

In [ ]:
# 1°) Montamos el Drive al colab , que es donde esta el archivo Zip que descargamos del link --->
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# 1°)
import gensim
print(gensim.__version__)


4.3.2


In [ ]:
# 2°) Descomprimimos el zip--->

#!unzip "/content/drive/MyDrive/MachineLearningAvanzado/SBW-vectors-300-min5.bin.gz"


In [ ]:
# 3°) Importamos biblioteca e Instanciamos el modelo --->

from gensim.models import KeyedVectors

# Ruta al archivo .bin.gz de embeddings
model = '/content/drive/MyDrive/MachineLearningAvanzado/SBW-vectors-300-min5.bin.gz'

# Cargar el modelo Word2Vec desde el archivo .bin.gz
modelo = KeyedVectors.load_word2vec_format(model, binary=True)


In [ ]:
# 4°) Vamos a entender el modelo un poco mas, vamos a ver como vamos a trabajar con nuestros KeyedVectors --->

#modelo.get_vector("rusia")
vector = modelo['rusia'] # Use indexing to get the vector for 'rusia'
print(vector)

[ 2.41386495e-03 -2.75877677e-02  2.90296692e-02 -1.11172304e-01
  1.18620127e-01 -1.82610556e-01 -1.28264762e-02  3.63720506e-02
  3.33251119e-01 -1.22887135e-01  1.48669794e-01  8.25086087e-02
 -5.95312472e-03 -2.79966325e-01  5.39358258e-02 -1.29495278e-01
  1.06282616e-02 -1.46060437e-01 -4.77864712e-01  2.65246592e-02
 -2.99940377e-01  8.78467709e-02  1.29306212e-01  1.45441636e-01
 -9.87920910e-02 -4.43036593e-02 -3.25560659e-01  7.84606040e-02
 -1.71662077e-01  2.80851662e-01  1.36841699e-01  1.38896421e-01
  1.98732410e-02 -2.04700112e-01 -7.12859929e-02  1.29271090e-01
  2.09103078e-01  1.12479992e-01 -2.87536591e-01  9.74789113e-02
 -6.69664741e-02  1.32165611e-01  7.61287585e-02 -2.50587672e-01
 -3.48826014e-02  1.66718867e-02  1.02963755e-02  4.98696081e-02
  1.42625734e-01  1.06565237e-01  1.50579974e-01  1.86168533e-02
 -1.20499834e-01  8.89115408e-03  7.72292465e-02  9.14911628e-02
 -2.83374697e-01  1.41645759e-01  8.03587064e-02 -2.72485651e-02
  4.97856215e-02 -8.63490

In [ ]:
# 5°) Le hacemos un LEN para ver la longitud que tiene el vector --->

#len(modelo.get_vector("rusia"))
# 400 posiciones
print(len(vector))

300


In [ ]:
# 6°) Como podemos comprobar este Vector y ver que es lo mas similar a Rusia, nos va a devolver las palabras que el considera
# se encuentra cercanas a rusia --->

modelo.most_similar("rusia") # Use most_similar method to find nearest neighbors


[('georgia', 0.6840266585350037),
 ('eeuu', 0.6735044717788696),
 ('sovietica', 0.6270561814308167),
 ('eslovenia', 0.6217913031578064),
 ('haiti', 0.6211448907852173),
 ('polonia', 0.6080873012542725),
 ('Kazakistán', 0.6035131812095642),
 ('Bielorussia', 0.5969798564910889),
 ('zelanda', 0.595187246799469),
 ('Prikhodko', 0.595004141330719)]

In [ ]:
# 7°) Si hacemos lo mismo con un marca de auto o un pais de Europa hace la misma relacion ---->

modelo.most_similar("peugeot") # Use most_similar method to find nearest neighbors


[('citroen', 0.8791922330856323),
 ('cdi', 0.8286747336387634),
 ('psa', 0.8272714018821716),
 ('tbn', 0.8142942786216736),
 ('xperia', 0.8097282648086548),
 ('kaDIGITO', 0.8069819211959839),
 ('daewoo', 0.8064446449279785),
 ('PHPSESSID', 0.8050963878631592),
 ('gcr', 0.8047224283218384),
 ('tkDIGITO', 0.7997612357139587)]

In [ ]:
palabras_similares = modelo.most_similar("portugal") # Use most_similar method to find nearest neighbors
print(palabras_similares)

[('morgado', 0.646475613117218), ('rsssf', 0.6181958317756653), ('gddc', 0.6166597008705139), ('valdes', 0.615239143371582), ('elcciones', 0.6151246428489685), ('unilex', 0.6145886182785034), ('deptos', 0.6131942272186279), ('skiandorra', 0.6126061677932739), ('tabless', 0.6114614009857178), ('espanya', 0.6110906004905701)]


###**EXPLORANDO RELACIONES ENTRE PALABRAS**

Cuando hablamos de una representación vectorial, nos preguntamos: ¿será que yo puedo hacer operaciones entre estos vectores? Entonces, si nosotros nos acordamos, por ejemplo, de nuestras clases de matemáticas, cuando estábamos aprendiendo álgebra lineal, que hablábamos sobre vectores y matrices, sabemos que podemos sí operar los vectores.

Vamos a utilizar un recurso que se llama **GeoGebra.rg**. Y vamos a entrar a esta página y vamos a utilizar la calculadora gráfica que ellos ofrecen. Entonces en esta calculadora gráfica pues yo tengo mi plano cartesiano, entonces supongamos que yo quiero establecer unos puntos en mi plano cartesiano, vengo a donde dice: herramientas y voy a colocar un punto. Entonces un punto.
Entonces seleccione una posición y colóquela. Entonces voy a colocar este punto aquí en esta posición. Y voy a seleccionar otro punto y voy a colocarlo en esta posición y voy a seleccionar a otro punto y voy a colocarlo, por ejemplo, en esta posición.

Si yo vengo a donde dice álgebra en la parte superior, entonces yo tengo mis puntos y sus respectivas coordenadas. Entonces, tengo un espacio bidimensional, o sea de dos dimensiones. El eje de las X y de las Y, entonces vamos a cambiar aquí el punto. Supongamos que esos puntos son países, entonces el primer país podría ser, por ejemplo, Ecuador.
Y el otro país podríamos decir que es Colombia. Y digamos que este que está aquí digamos que es por ejemplo Argentina. Entonces, ¿yo qué podría hacer? Yo podría, por ejemplo tomar los puntos que corresponden a Ecuador y a Colombia y sumarlos.

Entonces le doy a clic aquí donde dice + y le doy donde dice una expresión, entonces yo puedo decir simplemente Ecuador, que es un punto más Colombia. Y entonces me da un punto resultante, que sería éste de aquí que sería A. Yo también podría, si yo quiero hacer otra función aquí, una expresión, es decir, por ejemplo Ecuador – Colombia, y me mostraría otro punto.
Entonces, dependiendo de la operación que ejecute, yo voy a tener un resultado diferente al original, porque yo estoy digamos, tomando estos vectores y los estoy sumando y obtengo ese punto, y cuando los resto, obtengo este punto, entonces yo puedo operar los vectores entre sí. ¿Por qué hacemos esto?

Porque al hacer estas operaciones vamos a obtener vectores resultantes y estos vectores resultantes tienen la propiedad de que van a estar en la misma región. Entonces veamos en Python cómo podemos hacer esto con nuestros vectores.

In [ ]:
# 1°) Vemos como lo podemos hacer con PYTHON --->

modelo.most_similar(positive=["colombia", "ecuador"])

# Nos devuelve el grado de similaridad entra ellas en un orden decreciente.

[('america', 0.6284738183021545),
 ('uruguay', 0.6268236637115479),
 ('bucaramanga', 0.6031538844108582),
 ('atletico', 0.5954282879829407),
 ('postobon', 0.5931621193885803),
 ('carchi', 0.5929304957389832),
 ('bolivia', 0.5916794538497925),
 ('cartagena', 0.5915964245796204),
 ('cauca', 0.5915430784225464),
 ('sudamerica', 0.5883246660232544)]

In [ ]:
# 2°) Ahora tratemos de hacer esas compraciones entre objetos diferentes --->

modelo.most_similar(positive=["italia", "ferrari"])

# Nos devuelve semejanza, No es tanta ya que Ferrari es italiana marca de auto, no nos devuelve tanta semejanza.

[('milan', 0.7197256684303284),
 ('formulaDIGITO', 0.6900485754013062),
 ('atletico', 0.6868566870689392),
 ('bayern', 0.683440089225769),
 ('supersport', 0.682165801525116),
 ('uefa', 0.6795853972434998),
 ('whitney', 0.6774600744247437),
 ('dakar', 0.6753880381584167),
 ('audino', 0.6749749779701233),
 ('girone', 0.67469322681427)]

In [ ]:
# 3°) Ahora hacemos otro ejemlo pero con NEGATIVE que aleja la similiridad entre los Vectores que asignamos --->

modelo.most_similar(negative=["italia", "ferrari"])

# Nos devuelve similiridad que NO tiene sentido con Italia y Ferrari

[('Stormyran', 0.096382737159729),
 ('UWG', 0.08730211853981018),
 ('acuífero', 0.034277476370334625),
 ('acuíferos', 0.020184578374028206),
 ('segregado', 0.01361220795661211),
 ('segregada', 0.009701022878289223),
 ('drenan', 0.00539777148514986),
 ('aluvial', 0.0030193119309842587),
 ('MCP', 0.0027628433890640736),
 ('Trasvase', 0.0026574123185127974)]

***Con Negative nos devuelve cosas totalmente que no tienen sentido. Entonces ‘kenbugul’, ‘llenaba’, ‘armadura’, ‘protegía’, ‘liberaba’, ‘llenaban’, ‘oprimía’, ‘arrojaba’, ‘bubónicos’. Entonces aquí, yo por ejemplo al utilizar estas funciones de que si es para buscar más semejanza, o digamos buscar las palabras más distantes, entonces podemos nosotros operar los vectores y ellos van a encontrarse en una región, recordemos que estamos en un espacio vectorial de 400 dimensiones.***

***Entonces se puede hacer estas operaciones de modo que pueden hallarse estos patrones entre nuestros vectores.***

###**PROFUNDIZACION DE RELACIONES ENTRE PALABRAS**

Profundicemos un poco más en este tema tan interesante. Vamos a crear más líneas de código. Y, por ejemplo, vamos a crear también inclusive un pequeño texto aquí. Si yo digo por ejemplo, lo siguiente, ***la siguiente relación: nube : nubes :: estrella : estrellas***. Entonces, digamos si yo sé que de nube es a nubes como estrella es a estrellas, por ejemplo en mi cabeza, pues yo ya sé esta relación.

Yo sé que aquí estoy hablando de número, de coincidencia, que coinciden números. Entonces la nube es a nubes como estrella es a estrellas. Por ejemplo, otra. ***Carro : transporte :: casa ¿qué se les ocurre a ustedes? Vivienda, bueno. Por ejemplo, casa es a vivienda o carro es a transporte como casa es a vivienda, por ejemplo.***

Entonces, estas relaciones, ¿será que yo las puedo de alguna forma establecer a través de estos métodos vectoriales? Bueno, vamos a ver el ejemplo más detenidamente. Entonces, por ejemplo, si yo tengo el siguiente: modelo.most_similar. Entonces, si yo hago lo siguiente, si yo le hago positive, si yo sumo por ejemplo. So yo sumo los vectores por ejemplo ‘nube’ y ‘estrella’.
Y también le quiero restar por ejemplo, nubes. ¿Entonces, cuál sería mi vector resultante? Entonces vean ustedes una cosa. Volvamos aquí. Si yo pongo nube + estrellas - nube. ¿Ustedes qué se imaginan cuál debería ser la respuesta? Nubes. Perdón, nubes + estrella – nube. Ahora sí tiene sentido.

Entonces yo estoy sumando, digamos nubes, en plural, un sustantivo en plural más uno en singular, menos otro en singular, que tiene que ver con esta palabra. Entonces aquí automáticamente, y estoy viendo, es como un plural, buscando un plural. Mi respuesta, lo más lógico es que me diga que son estrellas.
Si ven la relación, yo tengo nubes, ver a una palabra en plural, más estrella, que es en singular menos el singular de nubes, que sería nube, entonces, lo más lógico es que yo tenga el plural de estrella.



In [ ]:
# 1°) Establecemos las relaciones entres los Vectores --->

modelo.most_similar(positive=["nubes", "estrellas"], negative=["nube"])

# El algoritmo encuentra esa relacion Plural.

[('supergigantes', 0.5424259305000305),
 ('astros', 0.5286556482315063),
 ('nebulosas', 0.515497624874115),
 ('eclipsantes', 0.5148696303367615),
 ('subenanas', 0.5124854445457458),
 ('estelar', 0.5027156472206116),
 ('supermasivas', 0.5004077553749084),
 ('luminosidades', 0.49870434403419495),
 ('enanas', 0.49432995915412903),
 ('Cefeidas', 0.4911353588104248)]

In [ ]:
# 2°) El Algoritmo tambien lo reconoce en genero --->

modelo.most_similar(positive=["mujer", "profesor"], negative=["hombre"]) # Le restamos negative.

# Nos devuelve genero Femenino

[('profesora', 0.8312232494354248),
 ('catedrática', 0.7350810766220093),
 ('Profesora', 0.7213646769523621),
 ('catedrático', 0.6825595498085022),
 ('doctorada', 0.6590282320976257),
 ('cátedra', 0.645167887210846),
 ('Catedrática', 0.6417908668518066),
 ('emérita', 0.6316346526145935),
 ('Profesor', 0.6310397386550903),
 ('doctora', 0.6257742643356323)]

In [ ]:
modelo.most_similar(positive=["mujer", "medico"], negative=["hombre"]) # Le resto negative.

# Le quitamos hombre y nos devuelve alguna profesiones asociadas a mujer y medico, pero tambien nos devuelve
# asociaciones que el algoritmo NO es capaz de hallar, como por ejemplo: "niñera", "amiga".

[('clinica', 0.5922838449478149),
 ('enfermera', 0.5733241438865662),
 ('partera', 0.5719038248062134),
 ('médico', 0.5637348294258118),
 ('obstetriz', 0.5626001954078674),
 ('ginecológica', 0.5597516894340515),
 ('tocólogo', 0.5474926829338074),
 ('gineco', 0.5469223856925964),
 ('obstetra', 0.5452975630760193),
 ('obstetricia', 0.539606511592865)]

***El modelo no es perfecto en primer lugar. Y por otro lado, el modelo tiene un bies, recordemos que estamos hablando de redes neuronales, en la red cada palabra, cada entrada tiene un peso y un bies asociado a ellas, y este bies entonces, de acuerdo a lo que se observa en el contenido con el cual se entrenan los modelos, hace que él no sea capaz de hallar estas relaciones.
Entonces, a veces por motivos de religión, de política, de género, de orientación de la persona, la parte de orientación sexual, todas estas cosas entonces el modelo puede tener un bies.***

***Entonces estos bieses digamos son asuntos de mucha relevancia en el área del procesamiento del lenguaje humano y hay mucha investigación al respecto para tratar de reducir este bies, porque finalmente los datos son el producto de lo que somos, de nuestra cultura, de lo que nosotros hemos ido construyendo como sociedad.***
***Entonces el bies siempre va a existir, pero la idea es que, en la medida que nuestros modelos evolucionen entonces, entonces este bies o este sesgo se disminuya. Entonces queríamos mostrar esta relación para que ustedes vean que de todas maneras, pues en el modelo, a pesar de ser muy preciso, pues no es perfecto.***

***Dato del Aula: SESGO EN LOS DATOS***

Como lo discutimos en el aula, la Inteligencia Artificial (IA) es una de las áreas con mayor potencial de transformar el modo como vivimos, la movilidad y la salud. Con el impulso y progreso de la IA durante los últimos años, ha sido posible el avance tecnológico en materia de hardware y un aumento exponencial de datos disponibles para entrenamiento de los modelos de aprendizaje de máquina.

Teniendo en cuenta que los modelos más promisorios en esta área dependen mucho de los datos que producimos, estos modelos pueden reproducir estereotipos de nuestra sociedad, impulsando aún más la desigualdad social. Desarrollar modelos que no reproduzcan los aspectos negativos de la sociedad es crítico para garantizar un futuro más justo para todos; por ello, áreas de investigación como ética en la IA han venido creciendo considerablemente en los últimos años.

Queremos recomendarte que leas este artículo presentado en el encuentro anual del Foro Económico Mundial, escrito por Richard Socher, científico jefe de Salesforce, que aborda de manera muy interesante los peligros involucrados en la reproductibilidad de estereótipos humanos en sistemas de IA.

Otra gran referencia bibliográfica es el capítulo 6, sección 6.11 - Bias and Embeddings, del libro Speech and Language Processing escrito por Dan Jurafsky y James H. Martin. En esta sección, ellos abordan el tema del sesgo, además de traer referencias con técnicas para reducir este problema en el embedding de palabras.

#<font color=red>**COMBINANDO VECTORES**</font>

---

###**TOKENIZACION**

* 1. Obtener noticias, ya fueron cargados nuetros dataset, utilizamos principalmente el titular.

* 2. Realizar la Vectorizacion de los titulares, primero Vectorizamos las palabras y luego las operamos para convertir otras frases en un solo vector.

* 3. Por ultimo hacemos la Clasificacion.

In [ ]:
# 1°) Hecemos LOC del titulo del entrenamiento y ponemos un numero para ver que titulo nos muestra --->

entrenamiento.titulo.loc[5643]

# Nos devuelve un titular: ¡ Cuanta humildad Eva ...! La captan comiendo...

'¡Cuánta humildad Eva Longoria! La captan comiendo taquitos de cochinita pibil en mercado de Mérida'

In [ ]:
# 2°) Ahora vamos a ver como la Categorizo el diario --->

entrenamiento.categoria.loc[5643]

# Nos devuelve la Ctegoria Mundo.

'mundo'

La idea es tomar nuestro titular y TOKENIZAR cada una de las palabras y asignarles un Vector y al hacer esto podemos operar nuestros Vectores entre si para obtener 1 solo Vector que represente ese titulo. Y con esa informacion vamos a poder proceder a entrenar nuestros modelos.

###**CONTRUYENDO EL TOKENIZADOR**

Vamos a tomar nuestras frases y la separamos por palabras (Tokenizar) y utiizamos NLTK.

In [ ]:
# 1°) Definimos una Funcion Tokenizador ---->

import nltk
nltk.download("punkt")
import string

def tokenizador(texto): # Recibe parametro texto y nos va a devolver una Lista con Token
  texto = texto.lower().replace("¡", " ").replace("¿", " ") # Tomamos palabras en minuscula solamente.
  lista_alfanumerica = [] # Lista vacia para almacenar palabras y nuemros

  for token_valido in nltk.word_tokenize(texto):
    if token_valido in string.punctuation:
      continue # Que continue si encuntra puntuaciones o signos
    lista_alfanumerica.append(token_valido)
  return lista_alfanumerica

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# 2°) Llamamos la funcion y colocamos el STRING del titulo de la noticia de mas arriba --->

tokenizador(entrenamiento.titulo.loc[5643])

# No remueve el sigo de apertura de Exclamacion, esto sucede porque al ser español no reconoce la funcion
# los signos de apertura que se usan solo en el español, para esto tenemos que agregar en la funcion
# puntualmente que signos queremos que nos borre ---> texto = texto.lower().replace("¡", " ").replace("¿", " ")

['cuánta',
 'humildad',
 'eva',
 'longoria',
 'la',
 'captan',
 'comiendo',
 'taquitos',
 'de',
 'cochinita',
 'pibil',
 'en',
 'mercado',
 'de',
 'mérida']

In [ ]:
# Si hacemos un print de la funcion STRING.PUNCTUATION podemos ver que signos reconoce --->

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


#<font color=red>**COMBINANDO VECTORES**</font>

---

Ya pudimos Vectorizar cada una de las palabras de nuestros textos, pero ahora tenemos que crear un VECTOR para que nos permita identificar todas las frases, un Vector de texto como tal, y el texto van a ser los titulares de las noticias.

###**SUMA DE VECTORES**

In [ ]:
# 1°) Importamos la biblioteca Numpy y vamos a definir una funcion de combinacion de vectores por suma
# y va a recibir Tokens --->

import numpy as np

def combinacion_vectores_por_suma(tokens):
  vector_resultante = np.zeros(300) # Matriz de 0 ceros de las 300 posiciones del modelo
  for token in tokens:
    try:
      vector_resultante += modelo.get_vector(token)
    except KeyError:
      continue
  return vector_resultante

In [ ]:
# 2°) Ahora creamos una variable palabra para pasar una frase de prueba y luego hacemos una combinacion de vectores
# por suma y colocamos nuestros Tokens --->

palabras = tokenizador("esta es una prueba de el tokenizador")
vector_texto = combinacion_vectores_por_suma(palabras)
print(len(vector_texto))
print(vector_texto)

# Sin el TRY/EXCEPT en la funcion nos devuelve un KeyError ya que la palabra TOKENIZADOR no esta en el vocabulario, esta
# palabra es una adaptacion del Ingles para el español, NO exite en el modelo de de Word Embbeding que usamos. Para evitar
# esto debemos agregar TRY/EXCEPT en la funcion.

# Luego de hacer esto nos devuelve los Vectores de esa frase.

300
[ 2.76417430e-01 -1.58912602e-01  1.12052033e+00 -5.80981908e-01
 -2.32586863e-01  7.06117500e-01  1.94477333e-01 -5.48061430e-02
  1.80467623e+00 -3.39351873e-01 -5.37948150e-01 -3.26756693e-01
  6.88175907e-01 -7.52478324e-01 -1.61008080e-01 -5.10633826e-01
  2.62305342e-01 -2.32391208e-01 -5.13517614e-01  1.01253449e+00
 -6.57654399e-01 -1.13863767e+00  4.43006668e-01  1.69649734e+00
 -4.86636002e-01 -9.83828921e-02  4.12313228e-01  1.04334123e+00
  4.96658640e-01  1.56617989e+00  7.66229037e-01 -6.01797348e-01
 -1.00906293e+00  3.70567856e-01  7.21053603e-01 -6.01695210e-01
 -4.53551553e-01 -4.35301573e-01 -8.57206687e-01 -1.49931391e-02
  2.42533080e-01  1.52306220e-01 -1.11986350e+00 -6.24655637e-01
 -7.66971007e-01 -1.03714629e+00  8.54260549e-02 -1.41196406e-01
  8.91030712e-01  1.60064513e+00 -4.69779670e-01  1.00350538e+00
  2.84894608e-01  2.05808887e+00 -1.08080504e-01  1.43592360e-01
 -8.55967004e-01 -1.22604439e+00  7.46420473e-02  7.70092987e-01
  6.69612825e-01  5.5

#<font color=red>**CLASIFICACION CON WORD2VEC**</font>

---

###**VECTORIZANDO BASE DE ENTRENAMIENTO Y TEST**

Vamos a crear nuestra Matriz con los textos, con esta matriz vamos a poder obtener nuestros respectivos Vectores para hacer nuestras Test y Entrenamiento.

In [ ]:
# 1°) Vamos a definir una funcion para crear nuestra Matriz y despues la aplicamos al dataset de Entrenamiento y Test de las noticias
# para de esta manera obtener vectores de frases --->

def matriz(textos):
  x = len(textos) # Longitud de los textos.
  y = 300
  matriz = np.zeros((x,y))
  for i in range(x):
    palabras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacion_vectores_por_suma(palabras_numeros)
  return matriz

# Esta funcion toma la longitud de los textos para los valores de x(cantidad de noticias) por 300 que son las dimensiones de
# nuestro modelo, creamos la matriz y a cada uno de los espacios de esa matriz le vamos a aplicar el Tokenizador para hacerlo
# a los textos y despues haciendo la combinacion de vectores por suma de cada uan de estas palabras Tokenizadas.


In [ ]:
# 2°) Ahora procedemos a crear nuestra Matriz de Vectores para Entrenamiento y Matriz de Vectores para Test ---->

matriz_train = matriz(entrenamiento.titulo)
matriz_test = matriz(test.titulo)

print(matriz_train.shape)
print(matriz_test.shape)

# Nos devuelve las filas y columnas.

(91844, 300)
(22961, 300)


***Dos matrices que ya tienen que tener 400 columnas las dos, y lo que va a variar es el número de filas de cada una de ellas.***

***Entonces va a ser una cantidad de filas para el entrenamiento que van a ser 90000 o algo así y la de pruebas, que va a ser 20000 o algo parecido. Entonces ella va a hacer esta separación y va a aplicarle lógicamente las funciones para hacer los tokens en cada una de las palabras, de los textos, y después combinar estos tokens a través de una suma vectorial de los mismos para obtener el titular con un vector asociado a ese titular.***

***Entonces él se va a demorar algunos momentos haciendo todo este esta ejecución. Y el próximo video entonces vamos a generar una baseline, vamos a utilizar un clasificador para poder tener como un marco de referencia y, posteriormente, utilizar un clasificador que será una regresión logística para entonces colocar a prueba nuestro modelo.***

***Noten ustedes que ya nos trajo el resultado, entonces la matriz de entrenamiento tiene 91844 filas por 400, y la de prueba tiene 22961 filas por 400 columnas. Entonces ya estamos listos para seguir a nuestro próximo paso, que es nuestro baseline, ya nos vemos.***



**Dato:**

***Métricas de evaluación***

Discutiremos algunas formas de evaluar un modelo de clasificación, detallando lo que es precisión (del inglés precision) y exhaustividad (del inglés recall). Sobre estos criterios de evaluación:

***¿Cuál es la diferencia entre precisión y exhaustividad?***

* La **Precisión** muestra el número de clases clasificadas correctamente sobre todo lo que fue clasificado como dicha clase, o sea, es la cantidad de verdadero-positivo sobre verdadero-positivo + falso-positivo. Por otro lado, la **Exhaustividad** es todo lo que fue clasificado correctamente sobre el total de aquella clase, es decir, la cantidad de verdadero-positivo sobre verdadero-positivo + falso-negativo. Entonces, la exhaustividad muestra la capacidad de recuperar clases relevantes, mientras que la precisión muestra qué tan bien estas clases son recuperadas.

 La precisión y la exhaustividad son métricas importantes en el proceso de clasificación, pues de nada sirve ser preciso sin ser completo.

###**BASELINE CON DUMMY CLASSIFIER**

Una vez que tenemos nuestras Matriz de Entrenamiento y Test listos vamos  acrear nuestra Baseline y para ello vamos a utilizar un recurso de SKLEARN llamado DUMMY CLASSIFIER que es un clasificador que nos permite tener como un marco de referencia para ver si el logra identificar algunos patrones y posteriormente con este Baseline vamos a poder colocar un clasificador como una REGRESSION LOGISTICA para tener un punto de comparacion.


In [ ]:
# 1°) Importamos el recurso, lo instanciamos y hacemos un fit --->

from sklearn.dummy import DummyClassifier

DC = DummyClassifier() #Instanciamos
DC.fit(matriz_train, entrenamiento.categoria) # x=titulo, y=categoria, de esta manera el va a aparender que cada uno de los titulares
# corresponde a una categoria especifica.
label_predict_dc = DC.predict(matriz_test) #Prediccion de la respectiva Categoria del Label que le va dar el modelo de Dummy.

In [ ]:
# 2°) Vamos a ver cuantas Categorias tenemos en el dataset de Prueba --->

test.categoria.unique()

# Nos devuelve 6 categorias.

array(['ciencia y tecnologia', 'entretenimiento', 'mundo', 'politica',
       'economia', 'deportes'], dtype=object)

In [ ]:
# 3°) Creamos un reporte que nos va a mostrar la Precision, Recall, todo el puntaje, media, todo el resultado de las Metricas para ver que
# tan acertado el es --->

from sklearn.metrics import classification_report

CR_dummy = classification_report(test.categoria, label_predict_dc) # Instanciamos, compara si el clasificador fue acertado o no.
print(CR_dummy)

# Con estos resultados nos arroja un WARNING:UNDIFINED METRIC WARNNING, nos dice que como hay 0.00 en algunos valores al momento de obtener
# estos resultados se van a ver afectados ya que NO debe haber 0.00 en el denominador para obtener de F1-score que es el resultado de la
# division de precission y recall, ya que esto tenderia a infinito y matematicamente es inviable.



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

ciencia y tecnologia       0.00      0.00      0.00      2864
            deportes       0.00      0.00      0.00      4277
            economia       0.24      1.00      0.38      5461
     entretenimiento       0.00      0.00      0.00      3580
               mundo       0.00      0.00      0.00      2838
            politica       0.00      0.00      0.00      3941

            accuracy                           0.24     22961
           macro avg       0.04      0.17      0.06     22961
        weighted avg       0.06      0.24      0.09     22961



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Definicion:**

***CBOW vs. Skip-Gram***

Sobre las diferencias entre las arquitecturas de entrenamiento CBOW y Skip-Gram, responde:

***¿Qué cambia en la manera de entrenar las representaciones vectoriales entre CBOW y Skip-Gram?***

* **CBOW** es entrenado para realizar la previsión de una determinada palabra teniendo el contexto como información de entrada. Por su parte, **Skip-Gram** necesita prever el contexto recibiendo como input una palabra. Ambos generan un vector denso de tamaño predeterminado.

 Esta es la diferencia de entrenamiento entre Skip-Gram y CBOW.

###**CLASIFICANDO CON REGRESION LOGISTICA**

Vamos a trabajar con este clasificador para poner a prueba nuestro modelo.

In [ ]:
# 1°) Importamos el clasificador, instanciamos, tiene una serie de parametros que vamos a intentar mantener lo mas estandar porsible, menos un parametro
# que vamos a modificar que es el numero de Iteraciones MAX_ITER, parque Wird2Vec pueda generalizar bien debe tener un numero alto de Iteraciones,
# y luego hacemos un Fit ---->

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter=1000) # 1000 iteraciones
LR.fit(matriz_train, entrenamiento.categoria)

# Nos devuelve que el modelo con 1000 iteraciones NO puede convergir, en mi caso si pudo ejecutarse el calsificador, puede ser por el que el modelo
# de Word Embbeding que esto utilizando tiene 300 posiciones y no 400 como la que nos dio el aula que no funcion el archivo.

LogisticRegression(max_iter=1000)

In [ ]:
# Probamos con 1500 Iteraciones --->

LR = LogisticRegression(max_iter=1500) # 1500 iteraciones
LR.fit(matriz_train, entrenamiento.categoria)

LogisticRegression(max_iter=1500)

**Dato:**

***Para saber más: regresión logística***

En esta aula utilizamos la Regresión Logística (RL) para realizar la clasificación de los titulares.



###**CLASIFICANDO CON REGRESION LOGISTICA PARTE 2**

Una vez que se logro Instanciar.

In [ ]:
# 1°) Vamos a ver de cuantas Iteraciones necesito el parametro MAX_ITER para poder convergir --->

LR.n_iter_

# Ese numero de iteraciones necesito para poder convergir, para poder generalizar.

array([398], dtype=int32)

In [ ]:
# 2°) Ahorar vamos a hacer un Label_Predict y le vamos a dar un PREDICT a los datos de Prueba(test) --->

label_predict = LR.predict(matriz_test)
CR = classification_report(test.categoria, label_predict)
print(CR)

                      precision    recall  f1-score   support

ciencia y tecnologia       0.63      0.56      0.59      2864
            deportes       0.70      0.73      0.72      4277
            economia       0.62      0.68      0.65      5461
     entretenimiento       0.58      0.60      0.59      3580
               mundo       0.59      0.60      0.59      2838
            politica       0.59      0.50      0.54      3941

            accuracy                           0.62     22961
           macro avg       0.62      0.61      0.61     22961
        weighted avg       0.62      0.62      0.62     22961



***CONCLUSION:***

Ahora sí obtuvimos unos resultados interesantes. Vean ustedes que en exactitud mi modelo tuvo 0.63 comparado a una exactitud de 0.24, o sea 63%, es muy buen resultado realmente, no es malo, o sea, cualquier cosa encima de 60 pues debe considerarse que no es tan malo, es así.

En ciencia y tecnología él tuvo una buena precisión. Recuerden ustedes que la precisión entonces disminuye el error tipo 1, o sea, el error de falsos positivos, o sea, generalizó bien, contra un recall de 0.56. Y también, por ejemplo, en deportes, entonces él más o menos que precisión y recall estuvo muy equilibrado, de nuestras 4272 noticias, en economía.

Priorizo recall, o sea hubo más error tipo 1, pero aquí mejoró el error tipo 2, digamos hubo más falsos negativos aquí, en este caso. Sin embargo, pues son valores muy cercanos, 0.61, 0.60, 0.61, 0.62. Y aquí en política fue donde no generalizó tan bien, tuvo un 59 contra un 49 de recall, un 59 contra un 49 de recall.

Entonces en términos generales, nosotros podemos decir que nuestro modelo generalizó comparativamente mucho mejor. Vean ustedes que nuevamente la exactitud fue el del 0.63, o sea 63% de exactitud, o sea de las 22961 titulares clasificó bien el 63%. Eso es bueno, la media macro, la media general estuvo también 62% tanto en precisión como recall. Muy bueno.

Y la media ponderada 63%. Entonces, ¿qué podemos concluir con este resultado? Que a través de una regresión logística, que es un modelo de los más robustos, porque él generaliza mejor y no es tan complejo, digamos, obtenemos resultados interesantes.

Podemos, lógicamente, en vez de utilizar en nuestra clasificación una combinación de vectores por suma, nosotros podríamos hacer n combinación de vectores, puede ser suma, cualquier tipo de operación que nosotros queramos realizar aquí para combinar nuestros vectores para obtener un vector resultante. Ya esto ya depende de lo que queramos hacer.

Y lo otro que también podríamos hacer es usar otros modelos, no solamente una regresión logística. Podríamos utilizar Support Vector Machine, podríamos utilizar Random Forest, podríamos utilizar árboles de decisión, podríamos utilizar todo tipo de clasificadores, bagging, stacking, en fin, todos los clasificadores que deseemos para ver qué modelo se adapta mejor.

También podríamos por ejemplo modificar algunos hiperparámetros aquí en nuestra regresión logística, no solamente el número máximo de iteraciones, sino otros parámetros, y así entonces podemos ir haciéndole un tunning a nuestro modelo para dejarlo mejor.

¿a costo de qué? Si nosotros nos vamos a colocar más precisos y vamos a digamos a querer que generalicen, digamos, tenga mejores resultados, podemos estar también sacrificando nuestro modelo al hacerlos tan exactos, porque al recibir nuevos datos, entonces él puede presentar más errores.

Pero lógicamente ahí es donde entra en juegos la ciencia de datos, la función del científico de datos para ver cómo puede mejorar su modelo y que él aún continúe generalizando bien, para los datos que nunca antes vio.
Entonces básicamente, en esto consiste nuestro curso inicial de procesamiento del lenguaje humano, utilizando Word2Vec.


**<font color=orage>CONCLUSION FINAL:</font>**

Concluimos con éxito nuestro entrenamiento, the Word2Vec, interpretación del lenguaje humano con Word Embedding. Entonces, cómo pudiste observar a lo largo de nuestro curso pudimos explorar un dataset, dos datasets, realmente, de noticias, uno con noticias ya procesadas para el entrenamiento y otro para realizar nuestras pruebas.

Utilizamos CountVectorizer para ver cómo funcionaba One-Hot-Encoding, creando vectores de palabras, colocando 1 cuando reconocía la palabra y 0 en los demás casos, pero que esto no era algo práctico o viable, porque si teníamos 50000 palabras, entonces digamos el vector iba a ser de una dimensión de 1 por 50000 y solamente un 1 y el resto 0 entonces esto no sería muy recomendable en términos de almacenamiento, por ejemplo.

También aprendimos a cargar un modelo Word2Vec utilizando digamos un archivo .zip que almacenamos en nuestro Google Drive. Los cargamos con gensim models, entendimos cómo funciona Word2Vec, cómo es que se generan estos vectores de palabras.

Recordemos que utilizamos un modelo de 400 dimensiones y pudimos sumarlas, restarlas, ver cómo es que se comportaba nuestro modelo. Bastante interesante todo este tema y aprendimos también a combinar estos vectores a través de suma. Un pre procesamiento para poder hacer tokens con estos vectores y después sumar esos tokens y combinarlos creando un solo vector para cada titular.

Y así entonces pudimos llegar a la clasificación de nuestros textos, generando una matriz de textos utilizando NumPy, lógicamente entrenamos digamos nuestra matriz, utilizando un DummyClassifier, inicialmente para generar un baseline y después una regresión logística, que nos permitió a nosotros obtener resultados relativamente interesantes. ¿Y qué podemos concluir de nuestro curso realmente?

Que mientras más datos haya, nuestro modelo va a generalizar mucho mejor. Por eso es tan importante que cuando nosotros vayamos a trabajar con Word Embedding tengamos muchos, muchos, muchos y muchos datos. Lo que hicimos fue con un dataset relativamente pequeño, de alrededor de 120000 noticias en total, pero si ustedes pueden trabajar con datasets más grandes, ustedes van a ver que él va a generalizar mucho mejor.

Y lo otro es que aún hay mucho por hacer en el área de Word Embedding en el idioma castellano y esto realmente es un factor limitante en todo lo que se podría hacer para generar otro tipo de modelos, trabajando con otro tipo de arquitecturas, pero básicamente este es nuestro curso.